# El propósito de este Notebook fue limpiar la base de datos donde vienen las estaciones del metro para poder usarla luego. 

La base de datos ya estaba bastante limpia. Básicamente limpié los nulos y luego me tomó un poco de tiempo separar todas las líneas con las que hace correspondencia una sola estación. El formato de la base limpia se ve hasta el final. Esta base se usará más adelante. 

In [ ]:
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore')


In [2]:
metro = pd.read_csv('estaciones-metro.csv', sep=";")

In [3]:
metro.head()

,Geometry,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,trip_heads,agency_id,Geopoint
0,"{""type"": ""Point"", ""coordinates"": [-99.17122364...",14124.0,NaN,Colegio Militar,Metro 2,19.448646,-99.171224,Cuatro Caminos - Tasqueña,METRO,"19.44864628,-99.17122364"
1,"{""type"": ""Point"", ""coordinates"": [-99.17658806...",14119.0,NaN,Chapultepec,Metro 1,19.420256,-99.176588,Observatorio - Pantitlán,METRO,"19.42025648,-99.17658806"
2,"{""type"": ""Point"", ""coordinates"": [-99.10934, 1...",14095.0,NaN,Apatlaco,Metro 8,19.378298,-99.109340,Garibaldi - Constitución de 1917,METRO,"19.378298,-99.10934"
3,"{""type"": ""Point"", ""coordinates"": [-99.08486724...",14234.0,NaN,Calle 11,Metro 12,19.320256,-99.084867,Tláhuac - Mixcoac,METRO,"19.32025578,-99.08486724"
4,"{""type"": ""Point"", ""coordinates"": [-99.08690572...",14061.0,NaN,Oceanía,Metro Línea 5 correspondencia con línea B.,19.445429,-99.086906,Pantitlán - Politécnico,METRO,"19.44542916,-99.08690572"


In [4]:
metro = metro.drop(columns=['stop_code']) #son puros vacíos

In [5]:
metro = metro.drop(columns=['agency_id']) #todos solo dicen 'METRO'; no aporta información

In [6]:
def clean_estaciones(x):
    x = str(x)
    if "correspondencia" in x:
        x = x[:14]
        return x
    return x
    

In [7]:
metro = metro.drop(columns=['Geometry'])

In [8]:
metro_limpio = metro.copy()

In [9]:
def clean_stations_1(x):
    """ Esta función me ayuda a obtener la línea principal de la que es parte la estación"""
    x = str(x)
    
    if "correspondencia" in x:
        x = x.split("correspondencia con")[0]
        x = x.replace("Línea", "")
        x = x.replace("  ", " ")
        return x.strip()
        
    elif "correspondencia" not in x:
        x = x.replace("Línea", "")
        x = x.replace("  ", " ")
        return x.strip()

In [10]:
lista_1 = metro_limpio.stop_desc.apply(clean_stations_1)

In [11]:
metro_limpio['estacion_principal'] = lista_1

In [12]:
def clean_stations_2(x):
    """Esta función me ayuda a obtener las líneas extras con las que se coordina esa estación."""
  
    x = str(x)
    
    if "correspondencia" in x:
        x = x.split("correspondencia con")[1]
        x = x.replace("Línea", "Metro")
        x = x.replace("línea", "Metro")
        x = x.replace(".", "")
        x = x.replace("  ", " ")
       
        
    elif "correspondencia" not in x:
        x = "Ninguna"
    
    return x.strip()

In [13]:
lista_2 = metro_limpio.stop_desc.apply(clean_stations_2)

In [14]:
metro_limpio['borrador_2'] = lista_2

In [15]:
def obtener_segunda_estacion(x):
    """De los metros que tienen 2 o más estaciones, quiero obtener la segunda estación exactamente."""

    x = str(x)
    x = x.replace("s", "")
    
    if x != "Ninguna":
        x = x[:7]
        
    return x.strip()
    
    

In [16]:
metro_limpio['estacion_2'] = metro_limpio['borrador_2'].apply(obtener_segunda_estacion)

In [17]:
def obtener_tercera_estacion(x):
    """Esta función es para obtener la tercera estación cuando así las haya"""
    x= str(x)
    x.replace("Metros 8", "Metro 8")
    
    if "y" not in x:
        x = "Ninguna"
    
    if x == 'Metros 5 9 y A':
        x = "Metro 9"
    
    if x == 'Metros 1 5 y 9':
        x = "Metro 5"
    
    if x == "Metros 1 9 y A":
        x = "Metro 9"
    
    if x == 'Metros 1 5 y A':
        x = "Metro 5"
    
    if "y" in x:
        x = x.split("y")[1]
        x = x.strip()
        
    return x 

In [18]:
metro_limpio['tercera_estacion'] = metro_limpio['borrador_2'].apply(obtener_tercera_estacion)

In [20]:
def obtener_cuarta_estacion(x):
    """Por último, esta función es para obtener la cuarta estación en caso de que las haya"""
    
    x= str(x)
    x.replace("Metros 8", "Metro 8")
    lista = ['Metros 5 9 y A', 'Metros 1 5 y 9', "Metros 1 9 y A", 'Metros 1 5 y A']
    
    if x in lista:
        x = "Metro" + x.split("y")[1]
    
    elif x not in lista:
        x = "Ninguna"
        
    return x.strip() 

In [21]:
cuarta_estacion = metro_limpio['borrador_2'].apply(obtener_cuarta_estacion)

In [22]:
metro_limpio['cuarta_estacion'] = cuarta_estacion

In [24]:
metro_limpio = metro_limpio.rename(columns={'estacion_principal': 'primera_estacion', 'estacion_2': 'segunda_estacion'})

In [25]:
metro_limpio = metro_limpio.drop(columns=['borrador_2'])

In [26]:
metro_limpio = metro_limpio.drop(columns=['stop_desc'])

In [ ]:
#metro_limpio[['primera_estacion', 'segunda_estacion', 'tercera_estacion', 'cuarta_estacion']][metro_limpio.cuarta_estacion != "Ninguna"]
#con eso se ve que quedaron bien las estaciones

In [30]:
metro_limpio = metro_limpio.drop(columns=['trip_heads', 'stop_id', 'Geopoint'])

In [31]:
metro_limpio = metro_limpio.rename(columns={'stop_name': 'nombre_estacion'})

In [32]:
metro_limpio.head()

,nombre_estacion,stop_lat,stop_lon,primera_estacion,segunda_estacion,tercera_estacion,cuarta_estacion
0,Colegio Militar,19.448646,-99.171224,Metro 2,Ninguna,Ninguna,Ninguna
1,Chapultepec,19.420256,-99.176588,Metro 1,Ninguna,Ninguna,Ninguna
2,Apatlaco,19.378298,-99.109340,Metro 8,Ninguna,Ninguna,Ninguna
3,Calle 11,19.320256,-99.084867,Metro 12,Ninguna,Ninguna,Ninguna
4,Oceanía,19.445429,-99.086906,Metro 5,Metro B,Ninguna,Ninguna


In [34]:
estaciones_limpias = metro_limpio.to_csv("estaciones_limpias_v1.csv")